# Import Libraries

In [50]:
import numpy as np
np.random.seed(42)

from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from art.estimators.classification.scikitlearn import ScikitlearnLogisticRegression
from art.attacks.inference.reconstruction import DatabaseReconstruction

from diffprivlib.models import LogisticRegression as DPLogisticRegression

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [35]:
X, y = load_iris(return_X_y=True)

In [36]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train

In [37]:
normal_model = LogisticRegression()
normal_model.fit(x_train, y_train)

LogisticRegression()

In [47]:
normal_estimator = ScikitlearnLogisticRegression(normal_model)

# Train - DP

In [39]:
dp_model = DPLogisticRegression()
dp_model.fit(x_train, y_train)

/home/alper/.local/lib/python3.10/site-packages/diffprivlib/models/logistic_regression.py:239: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "


LogisticRegression(accountant=BudgetAccountant(spent_budget=[(1.0, 0), (1.0, 0), (1.0, 0), (1.0, 0)]),
                   data_norm=11.11125555461668)

In [54]:
dp_estimator = ScikitlearnLogisticRegression(dp_model)

# Attack

In [42]:
target_row = int(np.random.random() * len(x_train))
target_row

114

In [44]:
x_train[target_row], y_train[target_row]

(array([5.4, 3.4, 1.7, 0.2]), 0)

In [45]:
x_train_e = np.delete(x_train, target_row, axis=0)
y_train_e = np.delete(y_train, target_row, axis=0)

# Database Reconstruction

### Normal Model

In [48]:
dr_normal = DatabaseReconstruction(estimator=normal_estimator)

In [ ]:
x_normal, y_normal = dr_normal.reconstruct(x_train_e, y_train_e)

In [59]:
rmse_normal = np.sqrt(((x_train[target_row] - x_normal) ** 2).sum() / len(x_train))
print("Inference RMSE:", rmse_normal)

Inference RMSE: 0.09874187473635555


In [53]:
print('Target Row:', x_train[target_row], ' - ', y_train[target_row])
print('Reconstructed Row:', x_normal, ' - ', np.argmax(y_normal))

Target Row: [5.4 3.4 1.7 0.2]  -  0
Reconstructed Row: [[4.60000114 3.60000333 1.00000326 0.20000159]]  -  0


### Differential Privacy Model

In [55]:
dr_dp = DatabaseReconstruction(estimator=dp_estimator)

In [56]:
x_dp, y_dp = dr_dp.reconstruct(x_train_e, y_train_e)

In [57]:
rmse_dp = np.sqrt(((x_train[target_row] - x_dp) ** 2).sum() / len(x_train))
print("Inference RMSE:", rmse_dp)

Inference RMSE: 0.09874210098795304


In [60]:
print('Target Row:', x_train[target_row], ' - ', y_train[target_row])
print('Reconstructed Row:', x_dp, ' - ', np.argmax(y_dp))

Target Row: [5.4 3.4 1.7 0.2]  -  0
Reconstructed Row: [[4.59999967 3.59999974 1.00000009 0.19999999]]  -  2
